In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch as pt
from pathlib import Path


class Stocks:
    def __init__(self, raw_data):
        self.data = pd.DataFrame(raw_data.T).stack().reset_index()
        self.data.columns = ['Day', 'Stock', 'Price']
        self.whatToGraph = ['Price']

    def raw(self):
        # Create directory if it doesn't exist
        output_dir = Path("Raw Data")
        output_dir.mkdir(parents=True, exist_ok=True)

        for id, stock_price in self.stocks_dict.items():
            plt.figure(figsize=(12, 6))
            plt.plot(stock_price, label=f'Stock {id}', linewidth=0.5)
            plt.title(f'Stock {id} Price')
            plt.xlabel('Day')
            plt.ylabel('Price')
            plt.legend(loc='upper left', fontsize='small')
            plt.grid(True)

            # Save the plot to the directory
            plot_path = output_dir / f'stock{id}.png'
            plt.savefig(plot_path)
            plt.close()

    def bbCalc(self, ma_period=21):
        # Calculate the moving average, standard deviation, and Bollinger Bands
        self.maCalc(ma_period)
        sd = self.data.groupby('Stock')['Price'].transform(lambda x: x.rolling(window=ma_period).std())
        self.data['Upper Band'] = self.data[f'{ma_period}MA'] + (sd * 2)
        self.data['Lower Band'] = self.data[f'{ma_period}MA'] - (sd * 2)

        data = [f'{ma_period}MA','Upper Band','Lower Band']
        for data in data:
            self.whatToGraph.append(data)

        return
    
    def maCalc(self, ma_period=21):
        self.data[f'{ma_period}MA'] = self.data.groupby('Stock')['Price'].transform(lambda x: x.rolling(window=ma_period).mean())
        
        self.whatToGraph.append(f'{ma_period}MA')

        return

    def rsiCalc(self, window=14):
        # calculate the price differences
        price_diff = self.data.groupby('Stock')['Price'].diff()

        # seperate gains and losses
        gain = price_diff.where(price_diff > 0, 0)
        loss = -price_diff.where(price_diff < 0, 0)

        avg_gain = gain.groupby(self.data['Stock']).transform(lambda x: x.rolling(window=window, min_periods=1).mean())
        avg_loss = loss.groupby(self.data['Stock']).transform(lambda x: x.rolling(window=window, min_periods=1).mean())

        rs = avg_gain/avg_loss

        rsi = 100 - (100 / (1+rs))

        self.data[f'RSI {window}'] = rsi

        self.whatToGraph.append('RSI')

        return

    def stochRSICalc(self, window=14):
        rsi_min = self.data[f'RSI {window}'].groupby(self.data['Stock']).transform(lambda x: x.rolling(window=window,min_periods=1).min())
        rsi_max = self.data[f'RSI {window}'].groupby(self.data['Stock']).transform(lambda x: x.rolling(window=window, min_periods=1).max())
        stoch_rsi = (self.data[f'RSI {window}'] - rsi_min) / (rsi_max - rsi_min) * 100

        self.data[f'StochRSI {window}'] = stoch_rsi

        self.whatToGraph.append(f'StochRSI {window}')
        return
    
    def macdCalc(self, slow_ema=26, fast_ema=12, signal=9):
        # long term ema
        self.data[f'{slow_ema}EMA'] = self.data.groupby('Stock')['Price'].transform(lambda x: x.ewm(span=slow_ema, adjust=False).mean())
        # short term ema
        self.data[f'{fast_ema}EMA'] = self.data.groupby('Stock')['Price'].transform(lambda x: x.ewm(span=fast_ema, adjust=False).mean())

        # calculate macd line
        self.data[f'MACD'] = self.data[f'{fast_ema}EMA'] - self.data[f'{slow_ema}EMA']

        self.data['MACD Signal'] = self.data.groupby('Stock')['MACD'].transform(lambda x: x.ewm(span=signal, adjust=False).mean())

        self.whatToGraph.append('MACD')
        self.whatToGraph.append('MACD Signal')

        return
    
    def dailyReturnyCalc(self):
        self.data['Daily Return'] = self.data.groupby('Stock')['Price'].pct_change()

        return
        
    def atrCalc(self, window=14):
        self.data['TR'] = self.data.groupby('Stock')['Price'].diff().abs()
        self.data['ATR'] = self.data.groupby('Stock')['TR'].transform(lambda x: x.rolling(window=window).mean())

        return

In [4]:
def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T
# read the data from the text file
file_path = './prices.txt'   
prcAll = loadPrices(file_path)
ma_period = 21
ema_period = 9
no_sd = 2

df = Stocks(prcAll)
df.dailyReturnCalc()
stock = df.data[df.data['Stock']==0]
print(stock)

Stock
0     0.000010
1    -0.000107
2    -0.000073
3    -0.000198
4     0.000016
5    -0.000490
6    -0.000102
7    -0.000019
8    -0.000011
9     0.000326
10   -0.000030
11   -0.000582
12   -0.000081
13   -0.000223
14   -0.000378
15    0.000116
16   -0.000095
17   -0.000063
18   -0.000339
19   -0.000353
20    0.000427
21    0.000012
22    0.000065
23   -0.000308
24   -0.000072
25    0.001148
26    0.000483
27   -0.000133
28    0.000004
29   -0.000171
30   -0.000454
31    0.000202
32   -0.000243
33   -0.000362
34   -0.000392
35    0.000635
36    0.000422
37    0.000072
38   -0.000113
39   -0.000032
40   -0.000115
41    0.000649
42   -0.000592
43    0.000107
44   -0.000234
45   -0.000092
46    0.000544
47   -0.000132
48   -0.000728
49   -0.000027
Name: Daily Return, dtype: float64
       Day  Stock  Price  Daily Return
0        0      0  13.46           NaN
50       1      0  13.48      0.001486
100      2      0  13.47     -0.000742
150      3      0  13.53      0.004454
200      4    